In [1]:
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import pickle

from scipy.interpolate import CubicSpline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error 
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tqdm import tqdm
import sys, os
import re

absolutepath = os.path.abspath('')
fileDirectory = os.path.dirname(absolutepath)

#Path of parent directory (moves outside of repository)
parentDirectory = os.path.dirname(fileDirectory)

In [8]:
train = 0.8
validation = 0.1
test = 0.1

ticker = 'MSFT'

In [4]:
ticker_df = pd.read_pickle(fileDirectory+ '/data/raw_data_2021-11-12_12.31.45.pkl')
coi = ['ticker','reportperiod','Open','High','Low','Close','Volume']
ticker_df['reportperiod'] = pd.to_datetime(ticker_df['reportperiod'])
ticker_df = ticker_df[ticker_df.ticker == ticker][coi]

    
# Need to shift high back to run apply function
# this doesn't work when all stocks are represented
ticker_df['Back_Shifted_Real_High'] = ticker_df['High'].shift(-1)

num_dates = ticker_df.shape[0]

In [9]:
train_dates = ticker_df['reportperiod'][ticker_df.ticker == ticker][6:round(train*num_dates)]
valid_dates = ticker_df['reportperiod'][ticker_df.ticker == ticker][6+round(train*num_dates):round((train+validation)*num_dates)]
test_dates =  ticker_df['reportperiod'][ticker_df.ticker == ticker][6+round((train+validation)*num_dates):]

In [10]:
test_dates

1496059   2019-09-26
1496058   2019-09-27
1496057   2019-09-30
1496056   2019-10-01
1496055   2019-10-02
             ...    
1495526   2021-11-05
1495525   2021-11-08
1495524   2021-11-09
1495523   2021-11-10
1495522   2021-11-11
Name: reportperiod, Length: 538, dtype: datetime64[ns]